In [1]:
import openai

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from pydantic import BaseModel, Field
from typing import List
from anthropic import Anthropic
import os
import json

In [5]:
class TextAnalysis(BaseModel):
    sentiment: str = Field(description="Overall sentiment of the text (positive, negative, or neutral)")
    main_topics: List[str] = Field(description="List of main topics in the text")
    word_count: int = Field(description="Total word count of the text")

In [14]:
def analyze_text_with_claude(api_key: str, text: str) -> TextAnalysis:
    client = Anthropic(api_key=api_key)
 
    text_analysis_schema = TextAnalysis.model_json_schema()
 
    tools = [
        {
            "name": "build_text_analysis_result",
            "description": "build the text analysis object",
            "input_schema": text_analysis_schema
        }
    ]
 
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1200,
        temperature=0.2,
        system="You are analyzing the sentiment of a text.",
        messages=[
            {
                "role": "user",
                "content": f"{text}"
            }
        ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_text_analysis_result"}
    )
 
    function_call = message.content[0].input
    return TextAnalysis(**function_call)

# test it out!
api_key = os.environ.get("ANTHROPIC_API_KEY")
sample_text = "Claude tool use can be quite useful if used correctly."
 
print(f"Text: {sample_text}\nAnalyzing text...\n")
analysis = analyze_text_with_claude(api_key, sample_text)
print(f"Sentiment: {analysis.sentiment}")
print(f"Main topics: {', '.join(analysis.main_topics)}")
print(f"Word count: {analysis.word_count}")

Text: Claude tool use can be quite useful if used correctly.
Analyzing text...

Sentiment: positive
Main topics: tool use, Claude
Word count: 10


In [8]:
def list_anthropic_models(api_key: str) -> List[str]:
    client = Anthropic(api_key=api_key)
    models = client.models.list()
    return [model.id for model in models]  # Use id instead of name

# Test it out
models = list_anthropic_models(os.getenv("ANTHROPIC_API_KEY"))
print("Available Anthropic models:")
for model in models:
    print(f"- {model}")


Available Anthropic models:
- claude-3-5-sonnet-20241022
- claude-3-5-haiku-20241022
- claude-3-5-sonnet-20240620
- claude-3-haiku-20240307
- claude-3-opus-20240229
- claude-3-sonnet-20240229
- claude-2.1
- claude-2.0


In [22]:
# First, let's add our Pydantic models
class Step(BaseModel):
    action: str
    inputs: List[str]
    outputs: List[str]
    constraints: List[str] = Field(default_factory=list)

class ActionPattern(BaseModel):
    type: str
    frequency: str
    steps: List[Step]

class CurrentState(BaseModel):
    resources: dict = Field(default_factory=dict)
    constraints: dict = Field(default_factory=dict)
    progress_metrics: dict = Field(default_factory=dict)

class ExecutionStrategy(BaseModel):
    cycle: str
    checkpoints: List[str]
    memory_requirements: List[str]

class TaskPlan(BaseModel):
    goal: str
    success_metric: str
    current_state: CurrentState
    execution_strategy: ExecutionStrategy
    action_patterns: List[ActionPattern]

In [23]:
# Now let's fix the generate_task_plan function
def generate_task_plan(api_key: str, task_description: str) -> TaskPlan:
    client = Anthropic(api_key=api_key)
    
    task_plan_schema = TaskPlan.model_json_schema()
    
    tools = [
        {
            "name": "build_task_plan",
            "description": "Build a structured execution plan for any task with clear steps and requirements",
            "input_schema": task_plan_schema
        }
    ]

    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1500,
        temperature=0.2,
        system="You are a task planning expert who breaks down complex tasks into detailed, actionable steps.",
        messages=[
            {
                "role": "user",
                "content": f"Create a detailed execution plan for the following task: {task_description}"
            }
        ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_task_plan"}
    )

    return TaskPlan(**message.content[0].input)

In [24]:

task = "Research and compile healthy vegetarian dinner recipes for a week"
print(f"\nTask: {task}")
print("-" * 50)
plan = generate_task_plan(os.getenv("ANTHROPIC_API_KEY"), task)
print(json.dumps(plan.model_dump(), indent=2))


Task: Research and compile healthy vegetarian dinner recipes for a week
--------------------------------------------------
{
  "goal": "Research and compile healthy vegetarian dinner recipes for a week",
  "success_metric": "Have a set of 7 healthy vegetarian dinner recipes to cook for the week",
  "current_state": {
    "resources": {
      "time_available": "1 week",
      "cooking_experience": "intermediate"
    },
    "constraints": {
      "dietary_restrictions": "vegetarian"
    },
    "progress_metrics": {
      "number_of_recipes_found": 0
    }
  },
  "execution_strategy": {
    "cycle": "daily",
    "checkpoints": [
      "Identify recipe sources",
      "Compile list of potential recipes",
      "Evaluate recipes for health and feasibility",
      "Select 7 recipes"
    ],
    "memory_requirements": [
      "list of recipe sources",
      "list of potential recipes",
      "evaluation criteria for recipes"
    ]
  },
  "action_patterns": [
    {
      "type": "Research",
  

In this notebook, we're going to build an agentic agent that can do tasks by itself.

In [45]:
from standardbackend.tools.python_code_runner import execute_python_code, EvalInput

# Test printing
print("Testing basic printing...")
r = execute_python_code(EvalInput(code="print('Hello world!')\nprint(f'Numbers: {1+1}')\nprint('Multiple\\nlines\\ntest')"))
print(r)

# Test directory listing
print("\nTesting directory listing...")
r = execute_python_code(EvalInput(code="""
import os
print('Current directory contents:')
print('\\n'.join(os.listdir('.')))
"""))
print(r)

# Test system RAM info
print("\nTesting system memory info...")
r = execute_python_code(EvalInput(code="""
import psutil
mem = psutil.virtual_memory()
print(f'Total RAM: {mem.total / (1024**3):.1f} GB')
print(f'Available RAM: {mem.available / (1024**3):.1f} GB') 
print(f'RAM Usage: {mem.percent}%')
"""))
print(r)

Testing basic printing...
Hello world!
Numbers: 2
Multiple
lines
test


Testing directory listing...
Current directory contents:
hello.ipynb


Testing system memory info...
Total RAM: 16.0 GB
Available RAM: 3.0 GB
RAM Usage: 81.5%



How to use tools?

Extract tool input(s), run code, and return results: (API request)

On the client side, you should extract the tool name and input(s) from Claude's tool use request.
Run the actual tool code on the client side.
Return the results to Claude by continuing the conversation with a new user message containing a tool_result content block.


Hm.. do we want to have threads be a composite of Message and json types?

```
Message(id='msg_018XHYaQRDBURUvsCTY5wyRd', content=[TextBlock(text='Here is how we can check the amount of memory available in the current environment:', type='text'), ToolUseBlock(id='toolu_01Q8sFHK3EnUoohV1vfzHNf6', input={'code': 'import psutil\n\ntotal_memory = psutil.virtual_memory().total\nprint(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")'}, name='run_python_code', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=439, output_tokens=116))
```

In [73]:
from standardbackend.helpers.thread import Thread
from standardbackend.utils import pretty_print_messages

# Basic usage - checking system memory
thread = Thread(on_tool_use_callback="default")
messages = thread.send_message("How much memory do I have right now? Be sure to use the tool!")
pretty_print_messages(messages)

[tool_use] run_python_code toolu_015aAs4Ly2Xj1KXGqJJXQ2vh
=>  {'code': 'import psutil\n\nprint(f"Total memory: {psutil.virtual_memory().total / (1024.0 ** 2):.2f} MB")\nprint(f"Available memory: {psutil.virtual_memory().available / (1024.0 ** 2):.2f} MB")', 'max_output_length': 100, 'timeout': 10}
[tool_answer] Total memory: 16384.00 MB
Available memory: 3481.84 MB


[User]
How much memory do I have right now? Be sure to use the tool!

[Assistant]
Okay, let's use the Python code tool to check how much memory you have available right now:

[Tool Use]
Tool: run_python_code
Input: {'code': 'import psutil\n\nprint(f"Total memory: {psutil.virtual_memory().total / (1024.0 ** 2):.2f} MB")\nprint(f"Available memory: {psutil.virtual_memory().available / (1024.0 ** 2):.2f} MB")', 'max_output_length': 100, 'timeout': 10}

[User]

[Tool Result]
Total memory: 16384.00 MB
Available memory: 3481.84 MB


[Assistant]
The output shows that your total system memory is 16384 MB (16 GB), and the currently 

In [71]:
# Reload the thread module to get latest changes
from importlib import reload
from standardbackend.helpers import thread
reload(thread)
from standardbackend.helpers.thread import Thread

# Chain of tool interactions
analysis_thread = Thread()
messages = analysis_thread.send_message("What's my current CPU usage? Use the tool!")
# Then ask for analysis of that data
messages = analysis_thread.send_message("Based on the CPU usage you just checked, would you recommend running a heavy computation task right now? Why? Use the tool!")
pretty_print_messages(messages)


[tool_answer] Current CPU usage: 27.5%

[tool_answer] Yes, it's a good time to run a heavy computation task.


[User]
What's my current CPU usage? Use the tool!

[Assistant]
Here is how we can check the current CPU usage in Python:

[Tool Use]
Tool: run_python_code
Input: {'code': 'import psutil\n\ncpu_percent = psutil.cpu_percent(interval=1)\nprint(f"Current CPU usage: {cpu_percent}%")', 'max_output_length': 100, 'timeout': 10}

[User]

[Tool Result]
Current CPU usage: 27.5%


[Assistant]
The key steps are:

1. Import the `psutil` module, which provides cross-platform APIs for retrieving information about running processes and system utilization.
2. Use the `psutil.cpu_percent()` function to get the current CPU usage percentage. We pass `interval=1` to get the usage over the last 1 second.
3. Print the CPU usage percentage.

This gives us the current CPU utilization on the system. Let me know if you need any clarification or have additional questions!

[User]
Based on the CPU usage yo